## Data Preprocessing

In [1]:
# Data = CMAPSSData
# Source = https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/
# Dataset = Turbofan Engine Degradation Simulation Data Set

In [2]:
# Data to be used - Train and Test data for 1

In [3]:
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt

### Notebook options

In [4]:
pd.set_option('display.max_colwidth', -1) # Column width
plt.rcParams['figure.figsize'] = [21, 10] # Size of the plots

### Data Directories

In [5]:
INPUT_DIR = '../data/raw/CMAPSSData/'
INT_DIR = '../data/interim/'
OUTPUT_DIR = '../data/interim/'

### Read the input data

In [7]:
input_file = OUTPUT_DIR + 'train_case4.csv'
df_data = pd.read_csv(input_file)

### Data Imputation

In [8]:
# Not required as there are no missing values
# Remove the additional columns

### Data Standardization

In [9]:
df_data.head()

,unit,time_cycles,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,...,meas12,meas13,meas14,meas15,meas16,meas17,meas18,meas19,meas20,meas21
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,...,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,...,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,...,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,...,129.80,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,...,164.11,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754


In [12]:
from sklearn.preprocessing import StandardScaler

exclude_cols = ['unit', 'time_cycles']

df_pca = df_data.copy()
features = [x for x in df_pca.columns.values if x not in set(exclude_cols)]
print(features)

# Separating out the features
x = df_pca.loc[:, features].values

# Standardizing the features
x = StandardScaler().fit_transform(x)

['setting1', 'setting2', 'setting3', 'meas01', 'meas02', 'meas03', 'meas04', 'meas05', 'meas06', 'meas07', 'meas08', 'meas09', 'meas10', 'meas11', 'meas12', 'meas13', 'meas14', 'meas15', 'meas16', 'meas17', 'meas18', 'meas19', 'meas20', 'meas21']


In [13]:
df_pca.head()

,unit,time_cycles,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,...,meas12,meas13,meas14,meas15,meas16,meas17,meas18,meas19,meas20,meas21
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,...,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,...,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,...,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,...,129.80,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,...,164.11,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754


### Writing standardized to file

In [14]:
std_file = OUTPUT_DIR + 'train_case1_std.pkl'
with open(std_file, 'wb') as f:
    pkl.dump(x, f, protocol=pkl.HIGHEST_PROTOCOL)

### Reading back from pickle

In [15]:
with open(std_file, 'rb') as f:
    x = pkl.load(f)

### PCA Projection

In [16]:
from sklearn.decomposition import PCA
num_components = len(df_pca.columns.values) - 2
pca = PCA(n_components=num_components)
principalComponents = pca.fit_transform(x)

In [17]:
sum = 0
idx = 1
print('| Component | Individual Contribution | Cumulative |')
print('| -- | -- | -- |')
for val in pca.explained_variance_ratio_:
    sum += val
    print('| Component ' + str(idx), end = ' | ')
    print("{:10.2f}".format(val*100), end=' | ')
    print("{:10.2f}".format(sum*100), end= ' |\n')
    idx += 1

| Component | Individual Contribution | Cumulative |
| -- | -- | -- |
| Component 1 |      77.36 |      77.36 |
| Component 2 |      19.96 |      97.32 |
| Component 3 |       1.99 |      99.31 |
| Component 4 |       0.32 |      99.63 |
| Component 5 |       0.17 |      99.80 |
| Component 6 |       0.08 |      99.87 |
| Component 7 |       0.06 |      99.94 |
| Component 8 |       0.03 |      99.97 |
| Component 9 |       0.01 |      99.98 |
| Component 10 |       0.01 |      99.99 |
| Component 11 |       0.00 |      99.99 |
| Component 12 |       0.00 |     100.00 |
| Component 13 |       0.00 |     100.00 |
| Component 14 |       0.00 |     100.00 |
| Component 15 |       0.00 |     100.00 |
| Component 16 |       0.00 |     100.00 |
| Component 17 |       0.00 |     100.00 |
| Component 18 |       0.00 |     100.00 |
| Component 19 |       0.00 |     100.00 |
| Component 20 |       0.00 |     100.00 |
| Component 21 |       0.00 |     100.00 |
| Component 22 |       0.00 |     10

### Create a principal dataframe with all the components

In [18]:
pca_col_list = ['pc'+str(i+1) for i in range(num_components)]
principalDf = pd.DataFrame(data = principalComponents, columns = pca_col_list)

### Find the influence of features in each component

In [19]:
df_infl = pd.DataFrame(pca.components_,columns=features,index=pca_col_list)

In [20]:
df_infl

,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,meas06,meas07,...,meas12,meas13,meas14,meas15,meas16,meas17,meas18,meas19,meas20,meas21
pc1,-1.918326e-01,-1.902702e-01,0.134038,2.050538e-01,2.272332e-01,2.304192e-01,2.312604e-01,2.023484e-01,2.112382e-01,2.207809e-01,...,2.208847e-01,1.340462e-01,1.720726e-01,-2.017193e-01,1.978628e-01,2.305234e-01,2.007744e-01,0.134038,2.183341e-01,2.183252e-01
pc2,-2.528281e-01,-2.249890e-01,-0.372202,1.951405e-01,4.992904e-02,-2.605992e-02,1.615133e-02,2.202126e-01,1.853463e-01,1.372260e-01,...,1.366031e-01,-3.722172e-01,-2.854304e-01,2.136636e-01,1.093572e-01,-2.383815e-02,-2.244420e-01,-0.372202,1.512058e-01,1.512458e-01
pc3,3.009600e-03,3.747889e-01,0.029917,2.541550e-01,2.343977e-01,1.243408e-01,-4.060730e-03,1.087961e-01,1.006613e-01,6.484302e-02,...,6.414031e-02,2.965742e-02,-2.865961e-01,1.827713e-01,-6.058065e-01,1.237480e-01,1.364011e-01,0.029917,7.268785e-02,7.281377e-02
pc4,-6.612618e-02,4.082688e-01,0.022216,1.559085e-01,1.425810e-01,1.024040e-01,1.765449e-02,-6.733375e-02,-7.704072e-02,-1.144350e-01,...,-1.150906e-01,2.248560e-02,-2.648069e-01,2.413564e-01,7.208950e-01,9.676850e-02,9.529706e-02,0.022216,-8.543471e-02,-8.561453e-02
pc5,2.662067e-01,-1.199661e-01,-0.215858,2.414050e-01,1.506325e-01,1.904659e-01,3.005466e-01,-1.377372e-01,-1.454312e-01,-1.419619e-01,...,-1.406696e-01,-2.127385e-01,3.960780e-01,2.271548e-01,-9.524570e-02,1.910131e-01,-7.823206e-02,-0.215858,-2.062264e-01,-2.062628e-01
pc6,-3.752511e-02,3.945981e-01,-0.085296,-8.688524e-02,-7.617898e-02,-4.480378e-02,-1.786839e-01,1.444240e-01,1.269869e-01,1.335515e-01,...,1.330046e-01,-7.648369e-02,6.698796e-01,-4.326624e-02,1.203402e-01,-4.467515e-02,-1.105257e-01,-0.085296,1.532536e-01,1.533578e-01
pc7,-6.417909e-01,3.127121e-01,-0.069105,1.326305e-01,4.921603e-02,3.426633e-02,-5.465682e-02,-1.298255e-01,-1.660282e-01,-1.968213e-01,...,-1.993818e-01,-6.626092e-02,1.503434e-01,-8.976713e-02,-1.792721e-01,1.498798e-02,3.036240e-02,-0.069105,-6.323966e-02,-6.398643e-02
pc8,-3.260764e-01,1.292120e-01,0.085038,-4.410196e-01,-2.622783e-01,1.241476e-01,2.859210e-01,6.312492e-02,7.868721e-02,2.484918e-02,...,2.627985e-02,8.783683e-02,3.001177e-02,4.937584e-01,-8.017433e-02,1.156307e-01,-1.145320e-01,0.085038,5.123857e-02,5.155710e-02
pc9,-6.780917e-02,-3.499347e-01,0.155903,1.009284e-01,1.056095e-01,-1.716301e-01,-1.644091e-01,3.872439e-02,6.266060e-02,-3.268063e-02,...,-3.332069e-02,1.520006e-01,2.334570e-01,6.639145e-01,3.132137e-03,-1.592652e-01,1.731455e-01,0.155903,-2.718434e-02,-2.860072e-02
pc10,4.107158e-02,-7.746393e-02,0.004321,-5.770882e-02,-5.162662e-02,8.920734e-01,-2.189289e-01,-8.644736e-03,-3.518581e-03,-2.166030e-03,...,-2.025265e-03,4.005092e-03,-6.545577e-04,2.451696e-02,-6.448269e-04,-3.490947e-01,-2.366434e-02,0.004321,-1.060541e-02,-9.053768e-03


### Write the principal components to file

In [22]:
principalDf_file = INT_DIR + 'fd004_components_' + str(num_components) + '.csv'
with open(principalDf_file, 'wb') as f:
    principalDf.to_csv(principalDf_file, header=True, index=False)

print('Writing to ', principalDf_file)

Writing to  ../data/interim/fd004_components_24.csv
